# !!This File Was Designed to Run on Google Colab!!

In [ ]:
!pip install transformers
!pip install pyspellchecker
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 16.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import pipeline
from spellchecker import SpellChecker

from tqdm.notebook import tqdm
from google.colab import drive

# 1 Load Data and Preprocess Data

In [ ]:
#### Load data
drive.mount("/content/gdrive")
# create paths for post and comment data
df_all = pd.read_csv("/Data_NLP/sentiment_manual_500.csv")
print(len(df_all))

Mounted at /content/gdrive
500


In [ ]:
#### Load sentiment analysis pipeline
classification_1 = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
classification_2 = pipeline("text-classification", model="sbcBI/sentiment_analysis_model")
classification_3 = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")
classification_4 = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#### Check dataframe
df_all.head(5)

,Unnamed: 0,manual,Original_Comment
0,0,-,The new one sucks very hard to work on and cos...
1,1,=,Oh for the Morris minor
2,2,-,"imagine being set on ""owning a Chevy EV."" i s..."
3,3,=,Parts are made in china..but India wants them ...
4,4,=,"Why, why, why do reviewers always test the top..."


In [ ]:
#### Split dataframe into test and target
df_target = df_all[["manual"]]
df_test = df_all[["Original_Comment"]]

In [ ]:
#### Preprocess Text Data
docs = df_test["Original_Comment"].to_list()

# 2 Test Sentiment Models

## Model 1

In [ ]:
#### Predict Results of Model 1
pred_1 = classification_1(docs)
pred_1[:5]

[{'label': '1 star', 'score': 0.6098779439926147},
 {'label': '3 stars', 'score': 0.33993053436279297},
 {'label': '3 stars', 'score': 0.3571414649486542},
 {'label': '3 stars', 'score': 0.3442147672176361},
 {'label': '2 stars', 'score': 0.413100004196167}]

In [ ]:
#### Get Labels of Predictions
pred_label_1 = ["neg" if d["label"] in ["1 star", "2 stars"] else
              "neu" if d["label"] in ["3 stars"] else
              "pos" for d in pred_1]
pred_dic_1 = {"Pred": pred_label_1}
df_pred_1 = pd.DataFrame(pred_dic_1)
# Map labels to the target labels' format
map_1 = {"neg":"-", "pos":"+", "neu":"="}
df_pred_1["Pred"] = df_pred_1["Pred"].map(map_1)
print(df_pred_1.head(5))

  Pred
0    -
1    =
2    =
3    =
4    -


In [ ]:
#### Evaluate Model Accuracy
print("acc:", accuracy_score(df_pred_1, df_target["manual"]))

acc: 0.47


## Model 2

In [ ]:
#### Predict Results of Model 2
pred_2 = classification_2(docs)
pred_2[:5]

[{'label': 'LABEL_0', 'score': 0.9571877717971802},
 {'label': 'LABEL_2', 'score': 0.41969120502471924},
 {'label': 'LABEL_2', 'score': 0.523595929145813},
 {'label': 'LABEL_0', 'score': 0.6555210947990417},
 {'label': 'LABEL_0', 'score': 0.9410003423690796}]

In [ ]:
#### Get Labels of Predictions
df_pred_2 = pd.DataFrame(pred_2)[["label"]]
# Map labels to the target labels' format
map_2 = {"LABEL_0":"-", "LABEL_1":"=", "LABEL_2":"+"}
df_pred_2["label"] = df_pred_2["label"].map(map_2)
print(df_pred_2.head(5))

  label
0     -
1     +
2     +
3     -
4     -


In [ ]:
#### Evaluate Model Accuracy
print("acc:", accuracy_score(df_pred_2, df_target["manual"]))

acc: 0.588


## Model 3

In [ ]:
#### Predict Results of Model 2
pred_3 = classification_3(docs)
pred_3[:5]

[{'label': 'LABEL_0', 'score': 0.9790450930595398},
 {'label': 'LABEL_1', 'score': 0.8122484683990479},
 {'label': 'LABEL_0', 'score': 0.6821998357772827},
 {'label': 'LABEL_1', 'score': 0.7748362421989441},
 {'label': 'LABEL_0', 'score': 0.8467355966567993}]

In [ ]:
#### Get Labels of Predictions
df_pred_3 = pd.DataFrame(pred_3)[["label"]]
# Map labels to the target labels' format
map_3 = {"LABEL_0":"-", "LABEL_1":"=", "LABEL_2":"+"}
df_pred_3["label"] = df_pred_3["label"].map(map_3)
print(df_pred_3.head(5))

  label
0     -
1     =
2     -
3     =
4     -


In [ ]:
#### Evaluate Model Accuracy
print("acc:", accuracy_score(df_pred_3, df_target["manual"]))

acc: 0.708


## Model 4

In [ ]:
#### Predict Results of Model 2
pred_4 = classification_4(docs)
pred_4[:5]

[{'label': 'negative', 'score': 0.9584051966667175},
 {'label': 'neutral', 'score': 0.6779246926307678},
 {'label': 'negative', 'score': 0.7291643023490906},
 {'label': 'neutral', 'score': 0.7124909162521362},
 {'label': 'negative', 'score': 0.8503708243370056}]

In [ ]:
#### Get Labels of Predictions
df_pred_4 = pd.DataFrame(pred_4)[["label"]]
# Map labels to the target labels' format
map_4 = {"negative":"-", "neutral":"=", "positive":"+"}
df_pred_4["label"] = df_pred_4["label"].map(map_4)
print(df_pred_4.head(5))

  label
0     -
1     =
2     -
3     =
4     -


In [ ]:
#### Evaluate Model Accuracy
print("acc:", accuracy_score(df_pred_4, df_target["manual"]))

acc: 0.712
